In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import statsmodels.formula.api as smf

In [4]:
cd "C:\Users\isom\99designs_project_brief_data_scraping\data_20220301"

C:\Users\isom\99designs_project_brief_data_scraping\data_20220301


In [27]:
# load data
df1 = pd.read_csv('cleaned_contest_data_after_change_part1.csv')
df2 = pd.read_csv('cleaned_contest_data_after_change_part2.csv')
df3 = pd.read_csv('cleaned_contest_data_after_change_part3.csv')
df4 = pd.read_csv('cleaned_contest_data_after_change_part4.csv')
df_before = pd.read_csv('cleaned_contests_before_change_v3.csv')

In [28]:
df_after = pd.concat([df1,df2])

In [29]:
df_after = pd.concat([df_after,df3])
df_after = pd.concat([df_after,df4])

In [30]:
df_before_did = df_before[['seeker','start_date','prize','number_of_participants','total_submission', 'number_of_submission_per_participants']].copy()

In [31]:
df_before_did.rename(columns={'number_of_participants':'solvers'},inplace=True)

In [32]:
df_after_did = df_after[['seeker','start_date','prize','total_submission', 'number_of_participants',
       'number_of_submission_per_participants']].copy()
df_after_did.rename(columns={'total_submission':'solvers'},inplace=True)

In [33]:
df_control = pd.read_csv('cleaned_contests_webpage.csv')

In [34]:
df_control_did = df_control[['seeker','start_date','prize','total_submission', 'number_of_participants',
       'number_of_submission_per_participants']].copy()
df_control_did.rename(columns={'total_submission':'solvers'},inplace=True)

In [35]:
# match the day in the control group
df_before_did = df_before_did[df_before_did.start_date>='2010-10-05'].copy()
df_after_did = df_after_did[(df_after_did.start_date>='2011-01-25') & (df_after_did.start_date<='2011-03-08')].copy()
before_after = pd.concat([df_before_did,df_after_did])

In [41]:
before_after

,seeker,start_date,prize,solvers,number_of_participants,number_of_submission_per_participants,treated,after
2589,Orders2,2010-10-05,300,107,32,3.343750,1,0
2590,Social Clown,2010-10-05,223,60,16,3.750000,1,0
2591,Tom Ham,2010-10-05,223,125,29,4.310345,1,0
2592,Scott D Photography,2010-10-05,223,243,34,7.147059,1,0
2593,Ryanpullano,2010-10-05,223,54,11,4.909091,1,0
...,...,...,...,...,...,...,...,...
104,Tim10016,2011-03-08,200,125,23,5.434783,1,1
105,shalom123,2011-03-08,200,97,18,5.388889,1,1
106,Bentwomey,2011-03-08,300,117,36,3.250000,1,1
107,Konstan,2011-03-08,227,56,8,7.000000,1,1


In [43]:
#gen df_control/treated
df_control_did['treated'] = 0
df_control_after = []
for i in df_control_did['start_date']:
    if i<='2011-01-24':
        df_control_after.append(0)
    else:
        df_control_after.append(1)
        
df_control_did['after'] = df_control_after

In [37]:
#gen before_after_1/treated
before_after['treated'] = 1
before_after_after = []
for i in before_after['start_date']:
    if i<='2011-01-24':
        before_after_after.append(0)
    else:
        before_after_after.append(1)
        
before_after['after'] = before_after_after

In [46]:
did_data = pd.concat([before_after,df_control_did])
did_data['treated_after'] = did_data['treated'] * did_data['after']

In [51]:
for i in did_data.columns[2:-3]:
    est = smf.ols(formula= i+' ~ treated + after + treated_after', data=did_data).fit()
    print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                  prize   R-squared:                       0.464
Model:                            OLS   Adj. R-squared:                  0.464
Method:                 Least Squares   F-statistic:                     1836.
Date:                Wed, 06 Apr 2022   Prob (F-statistic):               0.00
Time:                        21:16:50   Log-Likelihood:                -39848.
No. Observations:                6366   AIC:                         7.970e+04
Df Residuals:                    6362   BIC:                         7.973e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       644.5821      5.772    111.680

In [52]:
did_data.to_excel('did_data_not_aggregated.xlsx')